In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from fermat import Fermat
from sklearn.datasets import load_iris, load_digits
import networkx as nx
import igraph as ig
import neo4j


In [ ]:
X, y = load_iris(return_X_y=True, as_frame=True)
N = len(X)  # # observaciones
K = len(set(y))  # # clases


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    *load_iris(return_X_y=True, as_frame=True), test_size=0.2
)


In [ ]:
import scipy


In [ ]:
from sklearn.neighbors import KernelDensity
import numpy as np

# rng = np.random.RandomState(42)
runs = 200
res = np.empty(runs)
n, d = 1000, 3
test_size = 10


In [ ]:
int(1.2)


In [ ]:
for i in range(runs):
    kde = KernelDensity(kernel="gaussian", bandwidth=0.5).fit(X[:test_size])
    log_density = kde.score_samples(X)
    res[i] = log_density[:test_size].mean() > log_density[test_size:].mean()
res.mean()


In [ ]:
X_iris, _ = load_iris(return_X_y=True)
n, d = X_iris.shape
datasets = {"iris": X_iris, "unif(0, 1)": np.random.random_sample((n, d))}
(n, d), X_iris[:5]


Qué probabilidad $p$ hay de que en la densidad estimatada sea, en promedio, más alta en el _train set_ que en un _test set_ aún no visto?
Con un _test set_ pequeño en relación al de _train_, esperaría que la densidad esté bien estimada en todo el dominio de la X y entonces $p >= 1/2 + \delta$, con un $\delta$ "pequeño", ya que $E_{X_{train}}(\hat{f_X}(X)) \approx E_{X_{test}}(\hat{f_X}(X))$. Coun un test set "grande", de tamaño cercano a $n$, la estimación de densidad,e la densidad esté sobreajustada a los puntos conocidos, y siempre dé 


In [ ]:
def run(train_size, runs=100):
    results = {name: np.empty(runs) for name in datasets}
    for i in range(runs):
        for name, X in datasets.items():
            X_train, X_test = train_test_split(X, train_size=train_size)
            kde = KernelDensity(kernel="gaussian", bandwidth=0.5).fit(X_train)
            results[name][i] = (
                kde.score_samples(X_train).mean() > kde.score_samples(X_test).mean()
            )
    return pd.DataFrame(results)


 Ajustar bien la densidad es `facil` entrenando con "casi todos" los datos y `dificil` entrenando con "casi ninguno" de los datos.

In [ ]:
facil, dificil = 0.9, 0.1
runs = 100
favorable, adverso = run(facil, runs), run(dificil, runs)


In [ ]:
res = pd.concat(
    [favorable, adverso], keys=["favorable", "adverso"], names=["modo", "run"]
)


In [ ]:
res.groupby("modo").mean()


Bingo! En el contexto `favorable` de entrenamiento, la densidad estimada está más ajustada a los datos de _train_ que _test_, pero no siempre. En particular, el sobreajuste se nota más en el dataset real, `iris`, que en el sintético y "puramente aleatorio" `unif(0, 1)`.
En el contexto adverso, (casi) siempre la densidad estimada sobreajusta a train: en el dataset real "siempre" lo hace, mientras que en el sintético, alguna rara vez no se evidencia sobreajuste.

### Comparación de del humilde `EuclideanKDE` con la implementación de `scikit-learn`, `KernelDensity`

In [ ]:
norm_pdf = scipy.stats.norm.pdf


class EuclideanKDE:
    def __init__(self, kernel=norm_pdf, bandwidth=1):
        self.kernel = kernel
        self.bandwidth = bandwidth

    def fit(self, X):
        self.X_ = X
        self.n_, self.dim_ = X.shape
        return self

    def density(self, X, log=True):
        densities = (self.bandwidth**-self.dim_) * self.kernel(
            distance_matrix(X, self.X_) / self.bandwidth
        ).mean(axis=1)
        return np.log(densities) if log else densities


In [ ]:
X = datasets["iris"]


In [ ]:
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X)
sk_kde = KernelDensity(kernel="gaussian", bandwidth=1).fit(X)


In [ ]:
my_score = my_kde.density(X)
sk_score = sk_kde.score_samples(X)


In [ ]:
my_score[:5], sk_score[:5]


No se ven iguales pero pareciera haber una proporcionalidad

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
plt.plot(my_score, sk_score)
plt.axline([0, 0], slope=1, linestyle="dashed", color="gray")
est = LinearRegression().fit(X=sk_score.reshape(-1, 1), y=my_score)
plt.title(f"my_score = {est.intercept_.round(3)} + {est.coef_[0].round(3)} * sk_score")
plt.show()


Efectivamente, la relación pareciera de identidad, salvo por una sospechosa constante $\approx2.757$. Hmmmm. Al menos, parecieran funcionar igual. Revisando la documentación de [`KernelDensity.score_samples`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html?highlight=kerneldensity#sklearn.neighbors.KernelDensity.score_samples), leo:

    Returns 
        density: ndarray of shape (n_samples,)

        Log-likelihood of each sample in X. These are normalized to be probability densities, so values will be low for high-dimensional data.

Y en el código fuente, se lee (editado):

```python
        N = self.tree_.data.shape[0]
        log_density = self.tree_.kernel_density(X, h=self.bandwidth, kernel=self.kernel)
        log_density -= np.log(N)
        return log_density
```

Veamos si podemos revertir esa "normalización" de `log_density -= np.log(N)`:


In [ ]:
N = sk_kde.tree_.data.shape[0]
N, N == X.shape[0]


In [ ]:
atol_N = sk_kde.atol * N
log_density = sk_kde.tree_.kernel_density(
    X,
    h=sk_kde.bandwidth,
    kernel=sk_kde.kernel,
    atol=atol_N,
    rtol=sk_kde.rtol,
    breadth_first=sk_kde.breadth_first,
    return_log=True,
)


In [ ]:
assert np.allclose(sk_score + np.log(N), log_density)


In [ ]:
my_score[:5], log_density[:5]


In [ ]:
plt.plot(my_score, log_density)
plt.axline([0, 0], slope=1, linestyle="dashed", color="gray")
est = LinearRegression().fit(X=log_density.reshape(-1, 1), y=my_score)
plt.title(
    f"my_score = {est.intercept_.round(3)} + {est.coef_[0].round(3)} * log_density"
)
plt.show()


El problema, es que esa desnormalización, no es del factor esperado:

In [ ]:
np.log(N), LinearRegression().fit(X=sk_score.reshape(-1, 1), y=my_score).intercept_


Qué más hay de distinto? Habrán normalizado los datos sin avisar? La `data` (nuestra `X`) del estimador de densidad se accede desde `KernelDensity.tree_.data`, que es una `MemoryView` del array original.

In [ ]:
sk_data = np.asarray(sk_kde.tree_.data)
assert np.all(sk_data == X)


No, tampoco es eso. Bueno, ya se verá. Por el momento, son funcionalmente idénticas las implementaciones, salvo por esa constante.

In [ ]:
# TODO: review normalizing factors in BinaryTree.kernel_density
factor = 0.5 * d * np.log(2 * np.pi)
factor
# -factor - d * np.log(1)

### Comparación de Performance

In [ ]:
%%timeit
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X_digits)

In [ ]:
%%timeit
sk_kde = KernelDensity(kernel="gaussian", bandwidth=1).fit(X_digits)

`fit` es esencialmente "gratis", algunas pocas asignaciones en ambos casos.

In [ ]:
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X_digits)


In [ ]:
%%timeit
my_kde.density(X_digits)

In [ ]:
sk_kde = KernelDensity(kernel="gaussian", bandwidth=1).fit(X_digits)


In [ ]:
%%timeit
sk_kde.score_samples(X_digits)

In [ ]:
X_digits.shape


In [ ]:
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X_iris)


In [ ]:
%%timeit
my_kde.density(X_iris)

In [ ]:
sk_kde = KernelDensity(kernel="gaussian", bandwidth=1).fit(X_iris)


In [ ]:
%%timeit
sk_kde.score_samples(X_iris)

In [ ]:
X_digits.shape, X_iris.shape


Los resultados para `my_kde`, por ejemplo:
```
digits: 1.44 s ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
iris:  2.23 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
```

In [ ]:
1440 / 2.23, 1797 / 150


Ajustar `digits` toma 645 veces más tiempo que `iris`, con 12 veces más observaciones (pero claro, también 60 dimensiones más).


In [ ]:
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X_digits)


In [ ]:
%%timeit
my_kde.density(X_digits)

In [ ]:
%%timeit
my_kde.density(X_digits[:180])

Parece lineal en `n`.

In [ ]:
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X_digits[:, :8])


In [ ]:
%%timeit
my_kde.density(X_digits[:,:8])

In [ ]:
my_kde = EuclideanKDE(kernel=32, bandwidth=1).fit(X_digits[:, :16])


In [ ]:
%%timeit
my_kde.density(X_digits[:,:16])

In [ ]:
my_kde = EuclideanKDE(kernel=norm_pdf, bandwidth=1).fit(X_digits[:, :32])


In [ ]:
%%timeit
my_kde.density(X_digits[:,:32])

In [ ]:
n_dims = np.linspace(8, 64, 8).astype(int)
runs = 4

In [ ]:
import time

In [ ]:
results = []
for n_dim in n_dims:
    X = X_digits[:, :n_dim]
    my_kde = EuclideanKDE().fit(X)
    for i in range(runs):
        t0 = time.time()
        my_kde.density(X)
        t = time.time() - t0
        results.append({"n_dim": n_dim, "run": i, "time": t})

In [ ]:
res = pd.DataFrame(results)

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(x="n_dim", y="time", data=res)
est = LinearRegression().fit(res[["n_dim"]], res.time)
plt.title(
    f"time[s] = {est.intercept_.round(3)} + {est.coef_[0].round(3)} * n_dim"
)
plt.axline([0, est.intercept_], slope = est.coef_, linestyle="dashed", color="gray")
plt.show()

OK, también parece lineal en `n_dim`.

Los tiempos de `KernelDensity` son como la mitad de los de EuclideanKDE, pero no sé cuánto mejor/peor serían si implementase una DistanceMetric nueva para la distancia de Fermat. Por el momento, seguimos con la programacion habitual, y mantenemos los clasificadores de juguete.

In [ ]:
rang.random_sample((100, 3))


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from numpy import random as rnd
import seaborn as sns


In [ ]:
A = rnd.rand(100, 3)
A.sort(axis=1)
A = np.column_stack([np.zeros_like(A[:, 0]), A, np.ones_like(A[:, 0])])
a = A[0]
a, A[
    :5,
]


In [ ]:
def fermat_dist(x, alpha=1):
    """Fermat alpha-distance between `x_0` and `x_k`, in the line graph with nodes at `x = (x_1, ..., x_k)`."""
    return ((x[1:] - x[:-1]) ** alpha).sum()


In [ ]:
np.apply_along_axis(fermat_dist, axis=1, arr=X, alpha=3).mean()


In [ ]:
sample_size = 1000
scales = [1 / 10, 1 / 2, 1, 2, 10]
alphas = np.linspace(1, 4, 7)
ks = [1, 2, 5, 10]
results = []
for k in ks:
    A = rnd.rand(sample_size, k)
    A.sort(axis=1)
    A = np.column_stack([np.zeros_like(A[:, 0]), A, np.ones_like(A[:, 0])])
    for scale in scales:
        for alpha in alphas:
            results.append(
                dict(
                    k=k,
                    alpha=alpha,
                    scale=scale,
                    dists=np.apply_along_axis(
                        fermat_dist, axis=1, arr=scale * A, alpha=alpha
                    ),
                )
            )


In [ ]:
df = pd.DataFrame(results)


In [ ]:
df["mean_dist"] = df.dists.apply(np.mean)
df["scaled_dist"] = df.mean_dist / (df.scale**df.alpha)


In [ ]:
df.head()


In [ ]:
df[df.k == 1].pivot("alpha", "scale", "mean_dist").round(3)


In [ ]:
df[df.k == 1].pivot("alpha", "scale", "scaled_dist").round(3)


Está claro que si $c$ es la constance de escale `scale`, las distancias de fermat escalan según $c^{-\alpha}$. Ahora, cuánto cambian con el tamaño de muestra $k$? 

Para $k=1$, se puede calcular exactamente la esperanza de la longitud del camino. Si hay un único punto entre 0 y 1 elegido al azar según $X \sim \text{Unif}(0, 1)$, entonces la longitud del camino de Fermat cuando $alpha=2$ será $E\left(X^2 + (1-X)^2\right) = 2/3$, lo cual se ve en la tabla anterior. Para $k=1$ y otros valores de $\alpha$, la expresión no será tan bella pero es computable sin mucha dificultad. Para otros valores de $k$, sin embargo, ya entran en juego la distribución de los estadísticos de orden y no me resulta para nada evidente una fórmula cerrada.

Aproximémosla. Sean $X^{(0)} = 0, X^{(k+1)} = 1$ y $X^{(i)}, i=1,\dots,k$ las v.a. que surgen de ordenar una muestra $X_i \sim_{iid} \text{Unif}(0, 1), \ i\in[k]$. Para todo $k$ se cumple que cada "segmentito de recta", $\mathbb{E}\left(X^{(i+1)}-X^{(i)}\right) = 1 / (k + 1)$. Luego, esperaríamos que 

$$
\begin{align} dist_{\alpha}^k(0, 1) &= \mathbb{E}\ \left(\sum_{i=0}^k \left[X^{(i+1)}-X^{(i)} \right]^{\alpha}\right) \\
 &=  \sum_{i=0}^k\ \left( \mathbb{E}\left[X^{(i+1)}-X^{(i)} \right]^{\alpha}\right) \\
 &\approx (???) (k + 1) \left(\frac{1}{k+1}\right)^{\alpha}
   \end{align}

$$

Pero no vale que $E(X^k) = E(X)^k$! Qué se hace en su lugar? Hay que conocer la densidad de $X^{(i+1)}-X^{(i)}$ y calcularlo? Empíricamente, veamos como cambia la distancia con $k$.

In [ ]:
df[df.scale == 1].pivot("alpha", "k", "scaled_dist")


In [ ]:
df["k_scaled_dist"] = df.scaled_dist * ((df.k + 2) ** (df.alpha - 1))
df[df.scale == 1].assign().pivot("alpha", "k", "k_scaled_dist")


In [ ]:
df


In [ ]:
sample_size = 1000
alphas = [1.5, 1.75, 2, 2.25, 3, 4, 5]
ks = np.array([*range(1, 11), *rnd.choice(range(11, 2001), 50, replace=False)])

results = []
for k in ks:
    A = rnd.rand(sample_size, k)
    A.sort(axis=1)
    A = np.column_stack([np.zeros_like(A[:, 0]), A, np.ones_like(A[:, 0])])
    for alpha in alphas:
        results.append(
            dict(
                k=k,
                alpha=alpha,
                dists=np.apply_along_axis(fermat_dist, axis=1, arr=A, alpha=alpha),
            )
        )

df = pd.DataFrame(results)
df["mean_dist"] = df.dists.apply(np.mean)


In [ ]:
df[df.k < 50].pivot("alpha", "k", "mean_dist").round(3)


In [ ]:
plt.figure(figsize=(16, 7))
sns.lineplot(x="k", y="mean_dist", hue="alpha", data=df, marker="o")
plt.xscale("log")
plt.yscale("log")
plt.show()


Para la elección del ancho de banda $h$ es necesario saber la escala de las distancias! Con qué se come esto???

In [ ]:
results = []
for i in range(10000):
    s = rnd.rand(100)
    s.sort()
    dists = (np.array([s[0], *(s[1:] - s[:-1]), 1 - s[-1]])) ** 2
    results.append((dists.mean(), dists.std()))


In [ ]:
np.mean([std / mean for mean, std in results])


## Implementación KDEClassifier

In [ ]:
from scipy.stats import norm
from numpy.linalg import norm as euclidean_norm
from scipy.spatial import distance_matrix


In [ ]:
import scipy

scipy.spatial.distance.minkowski(A[0], A[2])


In [ ]:
repr(fermat)


In [ ]:
B = range(5)
C = range(7)
[[b * c for c in C] for b in B]


In [ ]:
(A[:] == A[2]).all(1)[:5]


In [ ]:
A.shape, np.zeros(10)


In [ ]:
# WIP - ignore
def euclidean_distance(x, y):
    return euclidean_norm(x - y)


class FermatDistance(Fermat):
    def __init__(self, **kwargs):
        super.__init__(**kwargs)

    def _fit(self, X):
        self.fit(X)
        self.X_ = X
        self.n_, self.d_ = X.shape
        self.is_fitted_ = True
        return self

    def __call__(A, B):
        if self.alpha == 1:
            return distance_matrix(A, B)
        else:
            return [[self._get_distance(a, b) for a in A] for b in B]

    def _get_distance(a, b):
        if not self.is_fitted_:
            self._fit()
        if any((self.X_[:] == a).all(1)):  # `a` is a node from X_
            to_X = np.zeros(self.n_)
        else:
            to_X = euclidean_distances(a.reshape(1, -1), verts)[0] ** self.alpha

        if a_known := np.where((X[:] == 2 * X[2]).all(1)):
            to_nodes = euclidean_distances(a.reshape(1, -1), verts)[0] ** alpha


In [ ]:
norm_pdf = scipy.stats.norm.pdf


class EuclideanKDE:
    def __init__(self, kernel=norm_pdf, bandwith=1):
        self.kernel = kernel
        self.bandwith = bandwith

    def fit(self, X):
        self.X_ = X
        self.n_, self.dim_ = X.shape
        return self

    def density(self, X, log=True):
        densities = (self.bandwith**-self.dim_) * self.kernel(
            distance_matrix(X, self.X_) / self.bandwith
        ).mean(axis=1)
        return np.log(densities) if log else densities


In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y  # , check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from scipy.special import softmax


class EuclideanKDEClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, kernel=norm_pdf, bandwith=1):
        self.kernel = kernel
        self.bandwith = bandwith

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        self.densities_ = {
            cls: EuclideanKDE(self.kernel, self.bandwith).fit(X[y == cls])
            for cls in self.classes_
        }
        return self

    def decision_function(self, X):
        return np.column_stack(
            [self.densities_[cls].density(X) for cls in self.classes_]
        )

    def predict(self, X):
        D = self.decision_function(X)
        return self.classes_[np.argmax(D, axis=1)]

    def predict_proba(self, X, temp=1):
        D = self.decision_function(X)
        return softmax(D / temp, axis=1)


In [ ]:
class FermatKDE:
    def __init__(self, kernel=norm_pdf, bandwith=1, alpha=2, path_method="FW"):
        self.kernel = kernel
        self.bandwith = bandwith
        self.alpha = alpha
        self.path_method = path_method

    def fit(self, X):
        self.X_ = X
        self.n_, self.dim_ = X.shape
        self.fermat_ = Fermat(self.alpha, self.path_method).fit(distance_matrix(X, X))
        self.distances_ = self.fermat_.get_distances()
        return self

    def density(self, X, log=True):
        densities = (self.bandwith**-self.dim_) * self.kernel(
            self.distances(X) / self.bandwith
        ).mean(axis=1)
        return np.log(densities) if log else densities

    def distances(self, X):
        distances_to_X = distance_matrix(X, self.X_) ** self.alpha
        return np.vstack([(d + self.distances_).min(axis=1) for d in distances_to_X])


In [ ]:
from scipy.special import softmax


class FermatKDEClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, kernel=norm_pdf, bandwith=1, alpha=2, path_method="FW"):
        self.kernel = kernel
        self.bandwith = bandwith
        self.alpha = alpha
        self.path_method = path_method

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        self.densities_ = {
            cls: FermatKDE(
                self.kernel, self.bandwith, self.alpha, self.path_method
            ).fit(X[y == cls])
            for cls in self.classes_
        }
        return self

    def decision_function(self, X):
        return np.column_stack(
            [self.densities_[cls].density(X) for cls in self.classes_]
        )

    def predict(self, X):
        D = self.decision_function(X)
        return self.classes_[np.argmax(D, axis=1)]

    def predict_proba(self, X, temp=1):
        D = self.decision_function(X)
        return softmax(D / temp, axis=1)


In [ ]:
A = np.array([[i] * 2 for i in range(3)])
Fermat(2, "FW").fit(distance_matrix(A, A)).get_distances()


In [ ]:
B = np.vstack([[-1, 0], [1, 0]])
B


In [ ]:
C = np.vstack([A, B])


In [ ]:
full = Fermat(2, "FW").fit(distance_matrix(C, C))
full_dists = full.get_distances()


In [ ]:
n_train = 120
train = FermatKDE(2, "FW").fit(A)
test_dists = train.distances(B)


In [ ]:
full_dists, test_dists


In [ ]:
full = Fermat(2, "FW").fit(distance_matrix(X, X))
full_dists = full.get_distances()


In [ ]:
n_train = 120
train = FermatKDE(2, "FW").fit(X[:n_train])
test_dists = train.distances(X[n_train:])


In [ ]:
np.where(full_dists[n_train:, :n_train] > test_dists + 0.001)


In [ ]:
full_dists[n_train:, :n_train][22, 101], test_dists[22, 101]


In [ ]:
pd.Series((full_dists[n_train:, :n_train] - test_dists).flatten()).describe()


In [ ]:
# Fermat distnaces between train obss, with X=train osbs, must be >= than
#   Fermat distances bw train obss, with X=(train obss + test obss)
assert (train.distances_ >= full_dists[:n_train, :n_train]).all()


```python
preds = []
fhats = []
for x in X_test:
    fhat = {}
    for cls in classes.keys():
        klass = classes[cls]
        verts, dists = klass["verts"], klass["dists"]
        n = verts.shape[0]
        to_verts = euclidean_distances(x.reshape(1, -1), verts)[0] ** alpha
        fmt_dists = [min(to_verts + dists[:, i]) for i in range(n)]
        # print(cls, np.mean(fmt_dists))
        fhat[cls] = (1 / h**D) * np.mean([kern(d / h) for d in fmt_dists])
    fhats.append(fhat)
    preds.append(pd.Series(fhat).argmax())
```

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier


In [ ]:
.shape

In [ ]:
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.90)
eucclf = EuclideanKDEClassifier().fit(X_train, y_train)
fmtclf = FermatKDEClassifier(alpha=2, bandwith=200).fit(X_train, y_train)
rfclf = RandomForestClassifier().fit(X_train, y_train)
rfclf2 = RandomForestClassifier().fit(
    np.concatenate([X_train, fmtclf.decision_function(X_train)], axis=1), y_train
)


In [ ]:
for clf in [eucclf, fmtclf, rfclf]:
    print(repr(clf), ": ", accuracy_score(clf.predict(X_test), y_test))
print(
    repr(rfclf2),
    " - enhanced: ",
    accuracy_score(
        rfclf2.predict(
            np.concatenate([X_test, fmtclf.decision_function(X_test)], axis=1)
        ),
        y_test,
    ),
)


In [ ]:
accuracy_score(eucclf.predict(X_test), y_test), accuracy_score(
    fmtclf.predict(X_test), y_test
)


In [ ]:
[x.shape for x in [X_train, X_test, y_train, y_test]]


In [ ]:
sp_softmax(D * 10, axis=1).round(3)


In [ ]:
(np.argmax(preds, axis=1) == y_test).mean()


In [ ]:
X, y = load_iris(return_X_y=True)
N = len(X)  # # observaciones
K = len(set(y))  # # clases


In [ ]:
kde = KDEstimator()


In [ ]:
kde.fit(X[y == 0])


In [ ]:
kde.predict(X[-1])


In [ ]:
y[0]


## TemplateClassifier

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances


class TemplateClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, demo_param="demo"):
        self.demo_param = demo_param

    def fit(self, X, y):

        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        # Return the classifier
        return self

    def predict(self, X):

        # Check if fit has been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        return self.y_[closest]


In [ ]:
unique_labels()
